In [1]:
!pip install pyspark==3.4.1
!pip install spark-nlp==5.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=543b2cc8a54d50439356223a1eb1e5c963f67ec7b4b70baa516ef0d711318422
  Stored in directory: /root/.cache/pip/wheels/e9/b4/d8/38accc42606f6675165423e9f0236f8e825f6b6b6048d6743e
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires pyspark[connect]~=3.5.1, but you have pyspark 3.4.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.6/547.6 kB 9

In [2]:
from pyspark.sql import SparkSession
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import Tokenizer, Normalizer, BertEmbeddings, SentimentDLModel, LemmatizerModel, UniversalSentenceEncoder
from pyspark.ml import Pipeline as MLPipeline

# Start Spark NLP session
import sparknlp
spark = sparknlp.start()

# Sample dataset (50 sample sentences)
sentences = [
    "I love this movie!", "This was the worst experience.", "Pretty decent overall.",
    "Absolutely fantastic!", "I'm not sure how I feel.", "Worst purchase ever.",
    "Great value for the money.", "It was okay, not great.", "Terrible, just terrible.",
    "Super fun and engaging!", "Would not recommend it.", "Kind of boring.",
    "Loved every minute!", "It was a disaster.", "Highly recommend!",
    "Too expensive for what you get.", "Amazing support team!", "Horrible food.",
    "I'll definitely buy it again.", "Meh, nothing special.", "Exceeded my expectations!",
    "Not worth the hype.", "Incredible storytelling.", "Never again.",
    "Pretty enjoyable!", "Worst customer service.", "Delightful and fresh.",
    "Disappointed.", "Can't wait to try it again!", "A total waste of time.",
    "Superb work!", "Very underwhelming.", "Loved the packaging.",
    "Felt very rushed.", "Unbelievably good.", "Didn't like it at all.",
    "Perfect execution!", "Mediocre at best.", "Simply amazing.",
    "It made my day!", "Forgettable.", "Super smooth experience.",
    "I regret buying it.", "Truly inspiring.", "Boring and repetitive.",
    "Five stars!", "Nothing new.", "Highly entertaining.", "Wasted potential.", "It was alright."
]

# Create DataFrame
df = spark.createDataFrame([(s,) for s in sentences], ["text"])

# Spark NLP pipeline
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

# Add sentence embeddings
use = UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

# Use pretrained sentiment model (Twitter-based)
sentiment_model = SentimentDLModel.pretrained("sentimentdl_use_twitter", "en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

# Create pipeline
pipeline = Pipeline(stages=[document_assembler, use, sentiment_model])

# Run the pipeline
result = pipeline.fit(df).transform(df)

# Show results
result.select("text", "sentiment.result").show(truncate=False)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]
+-------------------------------+----------+
|text                           |result    |
+-------------------------------+----------+
|I love this movie!             |[positive]|
|This was the worst experience. |[negative]|
|Pretty decent overall.         |[positive]|
|Absolutely fantastic!          |[positive]|
|I'm not sure how I feel.       |[negative]|
|Worst purchase ever.           |[negative]|
|Great value for the money.     |[positive]|
|It was okay, not great.        |[neutral] |
|Terrible, just terrible.       |[negative]|
|Super fun and engaging!        |[positive]|
|Would not recommend it.        |[negative]|
|Kind of boring.                |[negative]|
|Loved every minute!            |[positive]|
|It was a disaster.             |[negative]|
|Highly recommend!            

In [3]:
from transformers import pipeline
generator = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
prompt = "Artificial intelligence is transforming"

In [5]:
output = generator(prompt, max_length=50, num_return_sequences=1)
print("Step 2:Generated Text:\n", output[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Step 2:Generated Text:
 Artificial intelligence is transforming the world around us. We see it in everyday life, in the everyday work that we do in our homes and workplaces.

At least for now, artificial intelligence is not the only technology that can play a role in the world today. In fact, it is rapidly becoming one of the most important technologies to have in the world today.

In fact, we should take a moment to examine the future with this perspective. As the world becomes more and more connected to the world, the next generation of AI will have its own unique set of capabilities.

As we look to the future, we will see a new kind of AI with a new kind of capabilities.

We will see a new kind of AI with a new kind of capabilities.

From our perspective, the future is not only about AI, but also about the future of the world.

This means that we must look at the future with a new kind of view of the world.

Consider the world today.

We have a system of robots in place to help us t

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")


In [8]:
# Tokenize the prompt
tokens = tokenizer(prompt, return_tensors="pt")
print("\nToken IDs:\n", tokens['input_ids'][0].tolist())

NameError: name 'tokenizer' is not defined

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")

In [10]:
# Tokenize the prompt
tokens = tokenizer(prompt, return_tensors="pt")
print("\nToken IDs:\n", tokens['input_ids'][0].tolist())


Token IDs:
 [8001, 9542, 4430, 318, 25449]


In [11]:
with torch.no_grad():
    embeddings = model(**tokens).last_hidden_state
print("\nEmbeddings Shape:\n", embeddings.shape)



Embeddings Shape:
 torch.Size([1, 5, 768])


In [12]:
from IPython.display import Markdown
Markdown("""
### 🧠 Discussion Points

- **Tokenization**: Converts text into subword units and token IDs.
- **Embeddings**: Token IDs are mapped to dense vectors capturing meaning.
- **Transformer Layers**: Use attention to understand relationships between tokens.
- **Text Generation**: Predicts next words based on context and learned patterns.
""")



### 🧠 Discussion Points

- **Tokenization**: Converts text into subword units and token IDs.
- **Embeddings**: Token IDs are mapped to dense vectors capturing meaning.
- **Transformer Layers**: Use attention to understand relationships between tokens.
- **Text Generation**: Predicts next words based on context and learned patterns.


In [13]:
!pip install transformers torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 631.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 838.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 630.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 873.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [5]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load RoBERTa model fine-tuned for sentiment analysis
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Device set to use cpu


In [6]:
test_sentences = [
    "I absolutely love this!",
    "This is so frustrating and annoying.",
    "What a beautiful day!",
    "I can't stand this anymore.",
    "Totally worth it!",
    "Worst experience ever.",
    "I'm really happy with the results.",
    "This is not what I expected.",
    "Amazing job!",
    "Terrible service."
]

# True labels based on manual annotation
true_labels = [
    "positive", "negative", "positive", "negative", "positive",
    "negative", "positive", "negative", "positive", "negative"]


In [7]:
label_mapping = {'label_0': 'negative', 'label_1': 'neutral', 'label_2': 'positive'}
predicted_labels = [label_mapping[sentiment_pipeline(text)[0]['label'].lower()] for text in test_sentences]
print(predicted_labels)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


['positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative']


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate using multiclass metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print("Evaluation Metrics:")
print(f"Accuracy:  {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")

Evaluation Metrics:
Accuracy:  1.00
Precision: 1.00
Recall:    1.00
F1 Score:  1.00


In [9]:
from pyspark.sql import SparkSession
import pandas as pd
from transformers import pipeline
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [10]:
data = {
    "sentence": [
        "I love this product!", "This is the worst service ever.", "Absolutely fantastic experience.",
        "I'm not happy with the results.", "The movie was okay, not great.", "What a wonderful surprise!",
        "I would not recommend this.", "Such a delightful day.", "Terrible customer support.",
        "This phone is amazing!", "Very disappointing performance.", "I'm so excited about this!",
        "Could be better.", "Totally satisfied with my purchase.", "I hate how this works.",
        "It exceeded my expectations!", "Nothing special about it.", "I'm impressed by the quality.",
        "Worst purchase I've made.", "A pretty decent option."
    ],
    "label": [
        "positive", "negative", "positive", "negative", "neutral", "positive", "negative",
        "positive", "negative", "positive", "negative", "positive", "neutral", "positive",
        "negative", "positive", "neutral", "positive", "negative", "neutral"
    ]
}

In [11]:
spark = SparkSession.builder.appName("LLM Sentiment Evaluation").getOrCreate()


In [12]:
df_pd = pd.DataFrame(data)
df_spark = spark.createDataFrame(df_pd)

In [13]:
df = df_spark.toPandas()

In [14]:
classifier = pipeline("sentiment-analysis")  # Defaults to distilbert-base-uncased-finetuned-sst-2-english


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [15]:
def map_prediction(pred):
    label = pred['label'].lower()
    if label == 'positive':
        return 'positive'
    elif label == 'negative':
        return 'negative'
    else:
        return 'neutral'

df['predicted'] = df['sentence'].apply(lambda x: map_prediction(classifier(x)[0]))


In [16]:
print("\nClassification Report:")
print(classification_report(df['label'], df['predicted'], digits=3))

print("\nAccuracy Score:", accuracy_score(df['label'], df['predicted']))



Classification Report:
              precision    recall  f1-score   support

    negative      0.700     1.000     0.824         7
     neutral      0.000     0.000     0.000         4
    positive      0.900     1.000     0.947         9

    accuracy                          0.800        20
   macro avg      0.533     0.667     0.590        20
weighted avg      0.650     0.800     0.715        20


Accuracy Score: 0.8


In [17]:
df_result_spark = spark.createDataFrame(df)
df_result_spark.show(truncate=False)


+-----------------------------------+--------+---------+
|sentence                           |label   |predicted|
+-----------------------------------+--------+---------+
|I love this product!               |positive|positive |
|This is the worst service ever.    |negative|negative |
|Absolutely fantastic experience.   |positive|positive |
|I'm not happy with the results.    |negative|negative |
|The movie was okay, not great.     |neutral |negative |
|What a wonderful surprise!         |positive|positive |
|I would not recommend this.        |negative|negative |
|Such a delightful day.             |positive|positive |
|Terrible customer support.         |negative|negative |
|This phone is amazing!             |positive|positive |
|Very disappointing performance.    |negative|negative |
|I'm so excited about this!         |positive|positive |
|Could be better.                   |neutral |negative |
|Totally satisfied with my purchase.|positive|positive |
|I hate how this works.        

In [18]:
spark.stop()


In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from transformers import pipeline


In [20]:
spark = SparkSession.builder.appName("SparkLLMSentiment").getOrCreate()

In [21]:
data = [
    ("I love this product!",),
    ("This is the worst service ever.",),
    ("Absolutely fantastic experience.",),
    ("I'm not happy with the results.",),
    ("The movie was okay, not great.",),
    ("What a wonderful surprise!",),
    ("I would not recommend this.",),
    ("Such a delightful day.",),
    ("Terrible customer support.",),
    ("This phone is amazing!",),
    ("Very disappointing performance.",),
    ("I'm so excited about this!",),
    ("Could be better.",),
    ("Totally satisfied with my purchase.",),
    ("I hate how this works.",),
    ("It exceeded my expectations!",),
    ("Nothing special about it.",),
    ("I'm impressed by the quality.",),
    ("Worst purchase I've made.",),
    ("A pretty decent option.",)
]

columns = ["sentence"]
df = spark.createDataFrame(data, columns)

In [22]:
def load_model():
    return pipeline("sentiment-analysis")

def predict_sentiment(text):
    global clf
    if "clf" not in globals():
        clf = load_model()
    result = clf(text)[0]['label'].lower()
    return result

In [26]:
sentiment_udf = udf(predict_sentiment, StringType())
df_with_predictions = df.withColumn("predicted_sentiment", sentiment_udf("sentence"))
df_with_predictions.show(truncate=False)


+-----------------------------------+-------------------+
|sentence                           |predicted_sentiment|
+-----------------------------------+-------------------+
|I love this product!               |positive           |
|This is the worst service ever.    |negative           |
|Absolutely fantastic experience.   |positive           |
|I'm not happy with the results.    |negative           |
|The movie was okay, not great.     |negative           |
|What a wonderful surprise!         |positive           |
|I would not recommend this.        |negative           |
|Such a delightful day.             |positive           |
|Terrible customer support.         |negative           |
|This phone is amazing!             |positive           |
|Very disappointing performance.    |negative           |
|I'm so excited about this!         |positive           |
|Could be better.                   |negative           |
|Totally satisfied with my purchase.|positive           |
|I hate how th

In [27]:
spark.stop()